In [1]:
from binascii import unhexlify

unhexlify("76657273696f6e0000000000")

b'version\x00\x00\x00\x00\x00'

In [3]:
from helper import little_endian_to_int

little_endian_to_int(unhexlify('65000000'))

101

In [4]:
from binascii import hexlify
from helper import double_sha256

payload = unhexlify('721101000100000000000000bc8f5e5400000000010000000000000000000000000000000000ffffc61b6409208d010000000000000000000000000000000000ffffcb0071c0208d128035cbc97953f80f2f5361746f7368693a302e392e332fcf05050001')
hexlify(double_sha256(payload)[:4])


b'5f1a69d2'